In [1]:
import tensorflow as tf
batch_size = 128
(x_train, Y_train), (x_test, Y_test) = tf.keras.datasets.cifar10.load_data()

In [2]:
import numpy as np
all_digits = np.concatenate([x_train, x_test])
all_output = np.concatenate([Y_train, Y_test])
all_output_one_hot = tf.one_hot(np.squeeze(all_output), 10)
all_digits = all_digits.astype("float32") / 255.0

2021-12-02 21:09:18.912205: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:09:18.915327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:09:18.915599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:09:18.916146: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_output_one_hot))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.batch(batch_size).prefetch(1)

In [4]:
DATASET_SIZE=60000
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = dataset
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [6]:
model_16 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(32,32,3)),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
    ], name = "block_1"
)

base_model_1 = model_16

model_32 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(32,32,16)),
        tf.keras.layers.Conv2D(32, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
    ], name = "block_2"
)

base_model_2 = model_32

model_64 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(16,16,32)),
        tf.keras.layers.Conv2D(64, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(8,8), strides=(1, 1), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax")
    ], name = "block_3"
)

base_model_3 = model_64

In [7]:
class ForcedNet20(tf.keras.Model):
    def __init__(self, block_1, block_2, block_3):
        super(ForcedNet20, self).__init__()
        self.block_1 = block_1
        self.output_1 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(32,32), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(10, activation="softmax")
            ]
        )
        
        self.block_2 = block_2
        self.output_2 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(16,16), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(10, activation="softmax")
            ]
        )
        
        self.block_3 = block_3
    def compile(self, optimizer, loss_fn):
        super(ForcedNet20, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
    
    def train_step(self, data):
        images = data[0]
        labels = data[1]
        
        with tf.GradientTape(persistent=True) as tape:
            x = self.block_1(images)
            predictions_1 = self.output_1(x)
            
            x = self.block_2(x)
            predictions_2 = self.output_2(x)

            predictions_3 = self.block_3(x)
            
            loss_1 = self.loss_fn(labels, predictions_1)
            loss_2 = self.loss_fn(labels, predictions_2)
            loss_3 = self.loss_fn(labels, predictions_3)
        
        grads_1 = tape.gradient(loss_1, self.block_1.trainable_weights)
        grads_2 = tape.gradient(loss_2, self.block_2.trainable_weights)
        grads_final = tape.gradient(loss_3, self.block_3.trainable_weights)
        
        grads_1_output = tape.gradient(loss_1, self.output_1.trainable_weights)
        grads_2_output = tape.gradient(loss_2, self.output_2.trainable_weights)
                                
        self.optimizer.apply_gradients(
            zip(grads_1, self.block_1.trainable_weights,)
        )
        self.optimizer.apply_gradients(
            zip(grads_1_output, self.output_1.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2, self.block_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2_output, self.output_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_final, self.block_3.trainable_weights)
        )
        
        return {"Block_1_Loss": loss_1, "Block_2_Loss": loss_2, "Block_3_Loss": loss_3}

In [26]:
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import os

class Metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'mse': mean_squared_error(y_val, y_predict),
        })
        self._data.append({
            'mae': mean_absolute_error(y_val, y_predict),
        })
        self._data.append({
            'mape': mean_absolute_percentage_error(y_val, y_predict),
        })
        self._data.append({
            'accuracy': accuracy_score(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data

metrics = Metrics()

root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir(model_name):
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S") + " " + model_name
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir("ForcedLearning20")
callback = [tf.keras.callbacks.ReduceLROnPlateau(monitor='Block_3_Loss', patience=20, min_delta=0.0001,), tf.keras.callbacks.TensorBoard(run_logdir)]

ForcedLearner = ForcedNet20(model_16, model_32, model_64)
ForcedLearner.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=tf.keras.losses.CategoricalCrossentropy(),
)

history = ForcedLearner.fit(dataset, epochs=10000, callbacks=[callback])

Epoch 1/10000
469/469 [==============================] - 6s 9ms/step - Block_1_Loss: 1.9892 - Block_2_Loss: 1.9467 - Block_3_Loss: 1.9807 - lr: 3.0000e-04
Epoch 2/10000
469/469 [==============================] - 5s 10ms/step - Block_1_Loss: 1.7800 - Block_2_Loss: 1.6613 - Block_3_Loss: 1.6977 - lr: 3.0000e-04
Epoch 3/10000
469/469 [==============================] - 5s 10ms/step - Block_1_Loss: 1.6877 - Block_2_Loss: 1.5572 - Block_3_Loss: 1.5755 - lr: 3.0000e-04
Epoch 4/10000
469/469 [==============================] - 5s 10ms/step - Block_1_Loss: 1.6343 - Block_2_Loss: 1.4816 - Block_3_Loss: 1.4853 - lr: 3.0000e-04
Epoch 5/10000
469/469 [==============================] - 5s 10ms/step - Block_1_Loss: 1.6033 - Block_2_Loss: 1.4287 - Block_3_Loss: 1.4190 - lr: 3.0000e-04
Epoch 6/10000
469/469 [==============================] - 5s 10ms/step - Block_1_Loss: 1.5807 - Block_2_Loss: 1.3834 - Block_3_Loss: 1.3651 - lr: 3.0000e-04
Epoch 7/10000
469/469 [==============================] - 5s 10ms/

KeyboardInterrupt: 

In [16]:
inputs = tf.keras.Input(shape=(32,32,3))
x = model_16(inputs)
x = model_32(x)
outputs = model_64(x)
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(32,32,3)),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(16, 3, padding='same',activation='relu'),
        tf.keras.layers.Conv2D(32, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, strides=(2,2), padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(8,8), strides=(1, 1), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax")
    ], name = "vanilla_model"
)
run_logdir = get_run_logdir("VanillaNetwork")
callback = [tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=7), tf.keras.callbacks.TensorBoard(run_logdir)]
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))
history = model.fit(dataset, epochs=10000, callbacks=[callback])

In [25]:
model_temp = tf.keras.applications.MobileNetV2(weights=None, input_shape=(32,32,3), classes=10)
model_temp.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))
model_temp.summary()
model_temp.fit(dataset, epochs=10000, callbacks=[callback])

/home/jack/ml/my_env/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/10000
469/469 [==============================] - 11s 18ms/step - loss: 2.1744 - lr: 3.0000e-04
Epoch 2/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.8085 - lr: 3.0000e-04
Epoch 3/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.6072 - lr: 3.0000e-04
Epoch 4/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.4737 - lr: 3.0000e-04
Epoch 5/10000
469/469 [==============================] - 8s 18ms/step - loss: 1.3764 - lr: 3.0000e-04
Epoch 6/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.2980 - lr: 3.0000e-04
Epoch 7/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.2240 - lr: 3.0000e-04
Epoch 8/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.1612 - lr: 3.0000e-04
Epoch 9/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.0972 - lr: 3.0000e-04
Epoch 10/10000
469/469 [==============================] - 8s 17ms/step - loss: 1.

KeyboardInterrupt: 